In [1]:
from corpus_characterizer import generator_chunker
import numpy as np
import os
# from sklearn.metrics import mean_squared_error,mean_absolute_error, median_absolute_error, mean_squared_log_error, explained_variance_score, r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, explained_variance_score, \
    r2_score
import pandas as pd
from scipy.stats import describe, kurtosistest, skewtest, normaltest
from Conv1D_LSTM_Ensemble import pair_generator_1dconv_lstm_bagged
from Conv1D_ActivationSearch_BigLoop import pair_generator_1dconv_lstm #NOT BAGGED
from AuxRegressor import create_testing_set, create_training_set
import matplotlib.pyplot as plt 

# @@@@@@@@@@@@@@ RELATIVE PATHS @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Base_Path = "./"
image_path = "./images/"
train_path = "./train/"
test_path = "./test/"
analysis_path = "./analysis/"
chunker_path = analysis_path + "chunker/"
preds_path = analysis_path + "preds/"
load_sequence_aggregation_path = analysis_path + "load_sequence_agg/"
results_aggregation_path = analysis_path + "results_to_aggregate/"

#create the data-pair filenames (using zip), use the helper methods
train_set_filenames = create_training_set()
test_set_filenames = create_testing_set()
print(train_set_filenames)
print(test_set_filenames)

Using Theano backend.


after shuffling: [('sequence_2b_212_3_fv1b.npy', 'sequence_2b_212_3_label_fv1b.npy'), ('sequence_1b_75_6_fv1b.npy', 'sequence_1b_75_6_label_fv1b.npy'), ('sequence_1a_73_4.npy', 'sequence_1a_73_4_label_.npy'), ('sequence_2b_214_5_fv1b.npy', 'sequence_2b_214_5_label_fv1b.npy'), ('sequence_1b_52_3_fv1b.npy', 'sequence_1b_52_3_label_fv1b.npy'), ('sequence_1b_67_8_fv1b.npy', 'sequence_1b_67_8_label_fv1b.npy'), ('sequence_1d_142_3.npy', 'sequence_1d_142_3_label_.npy'), ('sequence_2c_267_8_fv1b.npy', 'sequence_2c_267_8_label_fv1b.npy'), ('sequence_1b_56_7_fv1b.npy', 'sequence_1b_56_7_label_fv1b.npy'), ('sequence_1a_26_7.npy', 'sequence_1a_26_7_label_.npy'), ('sequence_1b_71_2_fv1b.npy', 'sequence_1b_71_2_label_fv1b.npy'), ('sequence_2b_193_4_fv1b.npy', 'sequence_2b_193_4_label_fv1b.npy'), ('sequence_1b_64_5_fv1b.npy', 'sequence_1b_64_5_label_fv1b.npy'), ('sequence_2b_183_4_fv1b.npy', 'sequence_2b_183_4_label_fv1b.npy'), ('sequence_2c_270_1_fv1b.npy', 'sequence_2c_270_1_label_fv1b.npy'), ('seq

In [2]:
results_files_list = list(os.listdir(results_aggregation_path))
results_files_list.sort()
metrics_list = ['mae','mape','mse']
i=0
for files in test_set_filenames:
    #print("filename", files)
    i += 1
    temp_results_dict={}
    temp_results_df = pd.DataFrame(columns=metrics_list)
    data_load_path = test_path + '/data/' + files[0]
    label_load_path = test_path + '/label/' + files[1]
    # print("data/label load path: {} \n {}".format(data_load_path,label_load_path))
#     test_array = np.load(data_load_path)
#     test_label_array = np.load(label_load_path)[:, 1:]
    
    #nested for but whatever
    for results_file in results_files_list:
        print("analyzing ", str(files[0][:-4]), "with model ", str(results_file))
        complete_filename = results_aggregation_path + results_file
        interm_results_df = pd.read_csv(complete_filename,index_col='filename')
        #print(interm_results_df.index)
        print(interm_results_df.index.contains("sequence_1b_79_10_fv1b.npy"))
        
        suffix_check = interm_results_df.index.contains("sequence_1b_79_10_fv1b.npy")
        if suffix_check == True:
            seqname = str(files[0])
        if suffix_check == False:
            seqname = str(files[0])[:-4]
            
        temp_results_dict[str(results_file)]=interm_results_df.loc[seqname,:]
            
        for metric in list(interm_results_df.columns): #.remove('filename'): #make it adaptive to whatever results 
            if metric in temp_results_df.columns:
                temp_results_df.loc[:,[metric]] = interm_results_df.loc[seqname,[metric]]
        
    temp_results_df = pd.DataFrame.from_dict(temp_results_dict,orient='index')
    temp_results_df.sort_values(by=['mae'],inplace=True)
    temp_results_df.to_csv(load_sequence_aggregation_path + str(files[0][:-4]) + ".csv")

('analyzing ', 'sequence_2c_238_9_fv1b', 'with model ', 'combi_scores_et15_.csv')
True
('analyzing ', 'sequence_2c_238_9_fv1b', 'with model ', 'combi_scores_et2_model__tree_tinybidir_nodense_slowlr_relu_ca_sigmoid_da_3_cbd_standard_per_batch_sclr_l1_kr_HLR.csv')
True
('analyzing ', 'sequence_2c_238_9_fv1b', 'with model ', 'combi_scores_et30_.csv')
True
('analyzing ', 'sequence_2c_238_9_fv1b', 'with model ', 'combi_scores_et5_model__tree_tinybidir_nodense_slowlr_relu_ca_sigmoid_da_3_cbd_standard_per_batch_sclr_l1_kr_HLR.csv')
True
('analyzing ', 'sequence_2c_238_9_fv1b', 'with model ', 'combi_scores_et8_model__tree_tinybidir_nodense_slowlr_relu_ca_sigmoid_da_3_cbd_standard_per_batch_sclr_l1_kr_HLR.csv')
True
('analyzing ', 'sequence_2c_238_9_fv1b', 'with model ', 'combi_scores_linreg_.csv')
True
('analyzing ', 'sequence_2c_238_9_fv1b', 'with model ', 'combi_scores_rf15_.csv')
True
('analyzing ', 'sequence_2c_238_9_fv1b', 'with model ', 'combi_scores_rf2_model__tree_tinybidir_nodense_slo

('analyzing ', 'sequence_2c_238_9_fv1b', 'with model ', 'scores_conv__tree_medbidir_nodense_slowlr_relu_ca_tanh_da_3_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv')
False
('analyzing ', 'sequence_2c_238_9_fv1b', 'with model ', 'scores_conv__tree_mediumbidir_nodense_relu_ca_sigmoid_da_3_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv')
False
('analyzing ', 'sequence_2c_238_9_fv1b', 'with model ', 'scores_conv__tree_testmodel_relu_ca_sigmoid_da_3_cbd_minmax_per_batch_sclr_l1l2_kr_HLR.csv')
False
('analyzing ', 'sequence_2c_238_9_fv1b', 'with model ', 'scores_conv__tree_testmodel_relu_ca_sigmoid_da_3_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv')
False
('analyzing ', 'sequence_2c_238_9_fv1b', 'with model ', 'scores_conv__tree_testmodel_relu_ca_tanh_da_3_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv')
False
('analyzing ', 'sequence_2c_238_9_fv1b', 'with model ', 'scores_conv__tree_tinybidir_nodense_slowlr_relu_ca_sigmoid_da_3_cbd_standard_per_batch_sclr_l1_kr_HLR.csv')
False
('analyzing ', 'sequenc

('analyzing ', 'sequence_1b_88_9_fv1b', 'with model ', 'scores_conv__conv_nlstm_micro_relu_ca_sigmoid_da_3_cbd_standard_sclr_l1l2_kr_HLR.csv')
False
('analyzing ', 'sequence_1b_88_9_fv1b', 'with model ', 'scores_conv__conv_nlstm_micro_relu_ca_tanh_da_3_cbd_standard_sclr_l1l2_kr_HLR.csv')
False
('analyzing ', 'sequence_1b_88_9_fv1b', 'with model ', 'scores_conv__conv_nlstm_tiny_relu_ca_sigmoid_da_3_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv')
False
('analyzing ', 'sequence_1b_88_9_fv1b', 'with model ', 'scores_conv__conv_nlstm_tiny_relu_ca_sigmoid_da_3_cbd_standard_sclr_l1l2_kr_HLR.csv')
False
('analyzing ', 'sequence_1b_88_9_fv1b', 'with model ', 'scores_conv__conv_nlstm_tiny_relu_ca_tanh_da_3_cbd_standard_sclr_l1l2_kr_HLR.csv')
False
('analyzing ', 'sequence_1b_88_9_fv1b', 'with model ', 'scores_conv__convnormallstm_minmax_128b_relu_ca_tanh_da_3_cbd_l1l2_kr_HLR.csv')
False
('analyzing ', 'sequence_1b_88_9_fv1b', 'with model ', 'scores_conv__mape_minmax_`100dense_relu_ca_tanh_da_3_cbd

False
('analyzing ', 'sequence_1k_179_10', 'with model ', 'scores_conv__conv_micro_240dense_mae_highlr_elu_ca_tanh_da_3_cbd_minmax_per_batch_sclr_l1l2_kr_HLR.csv')
False
('analyzing ', 'sequence_1k_179_10', 'with model ', 'scores_conv__conv_micro_240dense_mapeelu_ca_tanh_da_3_cbd_minmax_per_batch_sclr_l1l2_kr_HLR.csv')
False
('analyzing ', 'sequence_1k_179_10', 'with model ', 'scores_conv__conv_micro_240dense_msle_highlr_0start_elu_ca_tanh_da_3_cbd_robust_per_batch_sclr_l1l2_kr_HLR.csv')
False
('analyzing ', 'sequence_1k_179_10', 'with model ', 'scores_conv__conv_micro_240dense_msle_highlr_0start_elu_ca_tanh_da_3_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv')
False
('analyzing ', 'sequence_1k_179_10', 'with model ', 'scores_conv__conv_micro_240dense_msle_highlr_elu_ca_tanh_da_3_cbd_minmax_per_batch_sclr_l1l2_kr_HLR.csv')
False
('analyzing ', 'sequence_1k_179_10', 'with model ', 'scores_conv__conv_micro_32dense_msle_rhiLR_0start_selu_ca_tanh_da_3_cbd_robust_per_batch_sclr_l1_kr_HLR.csv')

False
('analyzing ', 'sequence_2b_199_10_fv1b', 'with model ', 'scores_conv__conv_2lyr_mse_hilr_240d_orth_init_allregzd_start0_relu_ca_tanh_da_2_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv')
False
('analyzing ', 'sequence_2b_199_10_fv1b', 'with model ', 'scores_conv__conv_2lyr_msle_hilr_240d_orth_init_allregzd_start0_relu_ca_tanh_da_2_cbd_robust_per_batch_sclr_l1l2_kr_HLR.csv')
False
('analyzing ', 'sequence_2b_199_10_fv1b', 'with model ', 'scores_conv__conv_3lyr_msle_hilr_240d_orth_init_allregzd_start0_relu_ca_sigmoid_da_3_cbd_robust_per_batch_sclr_l1l2_kr_HLR.csv')
False
('analyzing ', 'sequence_2b_199_10_fv1b', 'with model ', 'scores_conv__conv_3lyr_msle_hilr_240d_orth_init_allregzd_start0_relu_ca_tanh_da_3_cbd_robust_per_batch_sclr_l1l2_kr_HLR.csv')
False
('analyzing ', 'sequence_2b_199_10_fv1b', 'with model ', 'scores_conv__conv_micro_120dense_mse_highlr_0start_elu_ca_tanh_da_3_cbd_robust_per_batch_sclr_l1l2_kr_HLR.csv')
False
('analyzing ', 'sequence_2b_199_10_fv1b', 'with model 

...

True
('analyzing ', 'sequence_2c_258_9_fv1b', 'with model ', 'combi_scores_et5_model__tree_tinybidir_nodense_slowlr_relu_ca_sigmoid_da_3_cbd_standard_per_batch_sclr_l1_kr_HLR.csv')
True
('analyzing ', 'sequence_2c_258_9_fv1b', 'with model ', 'combi_scores_et8_model__tree_tinybidir_nodense_slowlr_relu_ca_sigmoid_da_3_cbd_standard_per_batch_sclr_l1_kr_HLR.csv')
True
('analyzing ', 'sequence_2c_258_9_fv1b', 'with model ', 'combi_scores_linreg_.csv')
True
('analyzing ', 'sequence_2c_258_9_fv1b', 'with model ', 'combi_scores_rf15_.csv')
True
('analyzing ', 'sequence_2c_258_9_fv1b', 'with model ', 'combi_scores_rf2_model__tree_tinybidir_nodense_slowlr_relu_ca_sigmoid_da_3_cbd_standard_per_batch_sclr_l1_kr_HLR.csv')
True
('analyzing ', 'sequence_2c_258_9_fv1b', 'with model ', 'combi_scores_rf30_.csv')
True
('analyzing ', 'sequence_2c_258_9_fv1b', 'with model ', 'combi_scores_rf8_model__tree_tinybidir_nodense_slowlr_relu_ca_sigmoid_da_3_cbd_standard_per_batch_sclr_l1_kr_HLR.csv')
True
('analyz

('analyzing ', 'sequence_2c_258_9_fv1b', 'with model ', 'scores_conv__xgb_testmodel_relu_ca_tanh_da_3_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv')
False
('analyzing ', 'sequence_2c_258_9_fv1b', 'with model ', 'scores_conv__xgb_testmodel_smallbottleneck_relu_ca_tanh_da_3_cbd_minmax_per_batch_sclr_l1l2_kr_HLR.csv')
False
('analyzing ', 'sequence_2c_258_9_fv1b', 'with model ', 'scores_conv__xgb_testmodel_smallbottleneck_relu_ca_tanh_da_3_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv')
False
('analyzing ', 'sequence_2c_258_9_fv1b', 'with model ', 'scores_lstm_rerun__rerun_icacsis_l1l2_.csv')
False
('analyzing ', 'sequence_2c_258_9_fv1b', 'with model ', 'scores_lstm_rerun__rerun_icacsis_l1l2_4epochs_.csv')
False
('analyzing ', 'sequence_2c_258_9_fv1b', 'with model ', 'scores_lstm_rerun__rerun_icacsis_l1l2_longer_.csv')
False
('analyzing ', 'sequence_2c_258_9_fv1b', 'with model ', 'scores_lstm_rerun__test_safety_generator_.csv')
False
('analyzing ', 'sequence_1a_9_10', 'with model ', 'combi_s

('analyzing ', 'sequence_1a_9_10', 'with model ', 'scores_conv__convnormallstm_minmax_128b_relu_ca_tanh_da_3_cbd_l1l2_kr_HLR.csv')
False
('analyzing ', 'sequence_1a_9_10', 'with model ', 'scores_conv__mape_minmax_`100dense_relu_ca_tanh_da_3_cbd_l1_kr_HLR.csv')
False
('analyzing ', 'sequence_1a_9_10', 'with model ', 'scores_conv__mape_minmax_relu_ca_tanh_da_3_cbd_l1_kr_HLR.csv')
False
('analyzing ', 'sequence_1a_9_10', 'with model ', 'scores_conv__short_last_elu_ca_tanh_da_3_cbd_robust_per_batch_sclr_l1l2_kr_HLR.csv')
False
('analyzing ', 'sequence_1a_9_10', 'with model ', 'scores_conv__testnewadaptiveoutput_elu_ca_tanh_da_3_cbd_robust_per_batch_sclr_l1l2_kr_HLR.csv')
False
('analyzing ', 'sequence_1a_9_10', 'with model ', 'scores_conv__tree_medbidir_nodense_slowlr_relu_ca_sigmoid_da_3_cbd_standard_per_batch_sclr_l1_kr_HLR.csv')
False
('analyzing ', 'sequence_1a_9_10', 'with model ', 'scores_conv__tree_medbidir_nodense_slowlr_relu_ca_tanh_da_3_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv

False
('analyzing ', 'sequence_1c_28_9_fv1b', 'with model ', 'scores_conv__xgb_testmodel_smallbottleneck_relu_ca_tanh_da_3_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv')
False
('analyzing ', 'sequence_1c_28_9_fv1b', 'with model ', 'scores_lstm_rerun__rerun_icacsis_l1l2_.csv')
False
('analyzing ', 'sequence_1c_28_9_fv1b', 'with model ', 'scores_lstm_rerun__rerun_icacsis_l1l2_4epochs_.csv')
False
('analyzing ', 'sequence_1c_28_9_fv1b', 'with model ', 'scores_lstm_rerun__rerun_icacsis_l1l2_longer_.csv')
False
('analyzing ', 'sequence_1c_28_9_fv1b', 'with model ', 'scores_lstm_rerun__test_safety_generator_.csv')
False
('analyzing ', 'sequence_1k_189_10', 'with model ', 'combi_scores_et15_.csv')
True
('analyzing ', 'sequence_1k_189_10', 'with model ', 'combi_scores_et2_model__tree_tinybidir_nodense_slowlr_relu_ca_sigmoid_da_3_cbd_standard_per_batch_sclr_l1_kr_HLR.csv')
True
('analyzing ', 'sequence_1k_189_10', 'with model ', 'combi_scores_et30_.csv')
True
('analyzing ', 'sequence_1k_189_10',

('analyzing ', 'sequence_1k_189_10', 'with model ', 'scores_conv__short_last_elu_ca_tanh_da_3_cbd_robust_per_batch_sclr_l1l2_kr_HLR.csv')
False
('analyzing ', 'sequence_1k_189_10', 'with model ', 'scores_conv__testnewadaptiveoutput_elu_ca_tanh_da_3_cbd_robust_per_batch_sclr_l1l2_kr_HLR.csv')
False
('analyzing ', 'sequence_1k_189_10', 'with model ', 'scores_conv__tree_medbidir_nodense_slowlr_relu_ca_sigmoid_da_3_cbd_standard_per_batch_sclr_l1_kr_HLR.csv')
False
('analyzing ', 'sequence_1k_189_10', 'with model ', 'scores_conv__tree_medbidir_nodense_slowlr_relu_ca_tanh_da_3_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv')
False
('analyzing ', 'sequence_1k_189_10', 'with model ', 'scores_conv__tree_mediumbidir_nodense_relu_ca_sigmoid_da_3_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv')
False
('analyzing ', 'sequence_1k_189_10', 'with model ', 'scores_conv__tree_testmodel_relu_ca_sigmoid_da_3_cbd_minmax_per_batch_sclr_l1l2_kr_HLR.csv')
False
('analyzing ', 'sequence_1k_189_10', 'with model ', 's

In [3]:
#load the master_results file
master_results_df = pd.read_csv(analysis_path+"master_results_per_model.csv",index_col=0)
#declare a list of valid models, cutoff criteria mae < 1
model_ok_list = []
#if it meets the threshold, add to the list
for item in master_results_df.index:
    if master_results_df.loc[item,'mae_avg'] <= 0.50:
        model_ok_list.append(str(item))
#add an if statement to the

In [4]:
print("model ok list: ",model_ok_list, "length", len(model_ok_list))

('model ok list: ', ['combi_scores_et5_model__tree_tinybidir_nodense_slowlr_relu_ca_sigmoid_da_3_cbd_standard_per_batch_sclr_l1_kr_HLR.csv', 'combi_scores_rf8_model__tree_tinybidir_nodense_slowlr_relu_ca_sigmoid_da_3_cbd_standard_per_batch_sclr_l1_kr_HLR.csv', 'combi_scores_rf2_model__tree_tinybidir_nodense_slowlr_relu_ca_sigmoid_da_3_cbd_standard_per_batch_sclr_l1_kr_HLR.csv', 'combi_scores_et8_model__tree_tinybidir_nodense_slowlr_relu_ca_sigmoid_da_3_cbd_standard_per_batch_sclr_l1_kr_HLR.csv', 'combi_scores_et2_model__tree_tinybidir_nodense_slowlr_relu_ca_sigmoid_da_3_cbd_standard_per_batch_sclr_l1_kr_HLR.csv', 'scores_conv__xgb_testmodel_smallbottleneck_relu_ca_tanh_da_3_cbd_minmax_per_batch_sclr_l1l2_kr_HLR.csv', 'combi_scores_rf30_.csv', 'combi_scores_rf15_.csv', 'combi_scores_ridgesag_.csv', 'combi_scores_et30_.csv', 'combi_scores_ridgesparse_cg_.csv', 'combi_scores_ridgecholesky_.csv', 'combi_scores_et15_.csv', 'combi_scores_ridgesaga_.csv', 'combi_scores_ridgesvd_.csv', 'scores

In [5]:
main_colnames = ['seq_name','mse_avg','mse_med','mse_stdev','mae_avg','mae_med','mae_stdev','mae_normd_dev','mape_avg','mape_med','mape_stdev','msle_avg','msle_med','msle_stdev']
# mse_main_df = pd.DataFrame(columns=mse_main_colnames)
# mse_main_df.set_index(keys=['results_filename'], inplace=True)


sequence_results_per_model_list = list(os.listdir(load_sequence_aggregation_path))
#still need subset per model category... 
master_per_loadseq_df = pd.DataFrame(columns=main_colnames)
master_per_loadseq_df.set_index(keys=['seq_name'], inplace=True)

interm_per_loadseq_result_modeloksubset = pd.DataFrame(index=['model_name'])
#@@@@@@@@@@@@@@@@@@@@@@@@@@@
metrics_to_aggregate = ['mae','mse','mape','msle']
#@@@@@@@@@@@@@@@@@@@@@@@@@@@
for seq_results in sequence_results_per_model_list:
    interm_per_loadseq_result_df = pd.read_csv(load_sequence_aggregation_path + str(seq_results),index_col=0)
    interm_per_loadseq_result_modeloksubset = pd.DataFrame(index=['model_name']) 
    #set the index to be called model_name
    interm_per_loadseq_result_df.index.rename(name = 'model_name',inplace=True)
    
    #print('interm_per_loadseq_result_df columns',interm_per_loadseq_result_df.columns)
    
    #master_per_loadseq_df[str(seq_results)]
    for metric in metrics_to_aggregate:
        key_avg_colname = str(metric) + "_" + "avg"
        key_med_colname = str(metric) + "_" + "med"
        key_std_colname = str(metric) + "_" + "stdev"
        #print(interm_per_loadseq_result_df.index.isin([model_ok_list]))
        #print(interm_per_loadseq_result_df.loc[model_ok_list,'mae'])
        #for model in model_ok_list:
            #rint("model: ",model, "seq_results: ",seq_results) #needs two aggregations.. #subset models,THEN go.
        print("the mean", interm_per_loadseq_result_df.loc[model_ok_list,[str(metric)]].mean(skipna=True).values)
        print("the type of mean", type(interm_per_loadseq_result_df.loc[model_ok_list,[str(metric)]].mean(skipna=True)))
        print("the shape of mean", (interm_per_loadseq_result_df.loc[model_ok_list,[str(metric)]].mean(skipna=True)).shape)
        master_per_loadseq_df.loc[str(seq_results)[:-4],[key_avg_colname]] = interm_per_loadseq_result_df.loc[model_ok_list,[str(metric)]].mean(skipna=True).values
        master_per_loadseq_df.loc[str(seq_results)[:-4],[key_med_colname]] = interm_per_loadseq_result_df.loc[model_ok_list,[str(metric)]].median(skipna=True).values
        master_per_loadseq_df.loc[str(seq_results)[:-4],[key_std_colname]] = interm_per_loadseq_result_df.loc[model_ok_list,[str(metric)]].std(skipna=True).values
        
        print("the master df's entry:",master_per_loadseq_df.loc[str(seq_results)[:-4],[key_avg_colname]])
master_per_loadseq_df.sort_values(by=['mae_avg'],inplace=True)        
master_per_loadseq_df.to_csv(analysis_path + 'master_score_per_loadseq.csv')

('the mean', array([ 0.28518322]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mae_avg    0.285183
Name: sequence_2b_228_9_fv1b, dtype: object)
('the mean', array([ 0.24188164]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mse_avg    0.241882
Name: sequence_2b_228_9_fv1b, dtype: object)
('the mean', array([ 1295929.84803111]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mape_avg    1.29593e+06
Name: sequence_2b_228_9_fv1b, dtype: object)
('the mean', array([ nan]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", msle_avg    NaN
Name: sequence_2b_228_9_fv1b, dtype: object)
('the mean', array([ 0.16146705]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", m

('the shape of mean', (1,))
("the master df's entry:", mse_avg    0.0717966
Name: sequence_1b_58_9_fv1b, dtype: object)
('the mean', array([ 1293812.39517425]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mape_avg    1.29381e+06
Name: sequence_1b_58_9_fv1b, dtype: object)
('the mean', array([ nan]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", msle_avg    NaN
Name: sequence_1b_58_9_fv1b, dtype: object)
('the mean', array([ 0.15261317]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mae_avg    0.152613
Name: sequence_1k_189_10, dtype: object)
('the mean', array([ 0.06864688]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mse_avg    0.0686469
Name: sequence_1k_189_10, dtype: object)
('the mean', array([ 577835.19866669])

('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", msle_avg    NaN
Name: sequence_1d_119_10, dtype: object)
('the mean', array([ 0.18843965]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mae_avg    0.18844
Name: sequence_1l_79_10, dtype: object)
('the mean', array([ 0.11337155]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mse_avg    0.113372
Name: sequence_1l_79_10, dtype: object)
('the mean', array([ 718022.32304105]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mape_avg    718022
Name: sequence_1l_79_10, dtype: object)
('the mean', array([ nan]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", msle_avg    NaN
Name: sequence_1l_79_10, dtype: object)
('the 

('the mean', array([ 0.31122587]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mae_avg    0.311226
Name: sequence_1c_49_10_fv1b, dtype: object)
('the mean', array([ 0.28549703]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mse_avg    0.285497
Name: sequence_1c_49_10_fv1b, dtype: object)
('the mean', array([ 1300285.62578504]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mape_avg    1.30029e+06
Name: sequence_1c_49_10_fv1b, dtype: object)
('the mean', array([ nan]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", msle_avg    NaN
Name: sequence_1c_49_10_fv1b, dtype: object)
('the mean', array([ 0.15084202]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", m

Name: sequence_1k_178_9, dtype: object)
('the mean', array([ 0.06707655]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mse_avg    0.0670765
Name: sequence_1k_178_9, dtype: object)
('the mean', array([ 577967.1025344]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mape_avg    577967
Name: sequence_1k_178_9, dtype: object)
('the mean', array([ nan]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", msle_avg    NaN
Name: sequence_1k_178_9, dtype: object)
('the mean', array([ 0.1952289]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mae_avg    0.195229
Name: sequence_1k_219_10, dtype: object)
('the mean', array([ 0.11202902]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master d

...

("the master df's entry:", mse_avg    0.148669
Name: sequence_1d_138_9, dtype: object)
('the mean', array([ 578963.68415538]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mape_avg    578964
Name: sequence_1d_138_9, dtype: object)
('the mean', array([ nan]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", msle_avg    NaN
Name: sequence_1d_138_9, dtype: object)
('the mean', array([ 0.14797137]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mae_avg    0.147971
Name: sequence_1k_179_10, dtype: object)
('the mean', array([ 0.0669856]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mse_avg    0.0669856
Name: sequence_1k_179_10, dtype: object)
('the mean', array([ 577863.05057992]))
('the type of mean', <class 'pandas.core.serie

("the master df's entry:", mse_avg    0.112043
Name: sequence_1k_218_9, dtype: object)
('the mean', array([ 621363.84934918]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mape_avg    621364
Name: sequence_1k_218_9, dtype: object)
('the mean', array([ nan]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", msle_avg    NaN
Name: sequence_1k_218_9, dtype: object)
('the mean', array([ 0.15105142]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mae_avg    0.151051
Name: sequence_1c_19_10_fv1b, dtype: object)
('the mean', array([ 0.06592448]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mse_avg    0.0659245
Name: sequence_1c_19_10_fv1b, dtype: object)
('the mean', array([ 1293789.106701]))
('the type of mean', <class 'pandas.co

("the master df's entry:", mape_avg    578019
Name: sequence_1l_29_10, dtype: object)
('the mean', array([ nan]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", msle_avg    NaN
Name: sequence_1l_29_10, dtype: object)
('the mean', array([ 0.19119918]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mae_avg    0.191199
Name: sequence_1d_128_9, dtype: object)
('the mean', array([ 0.11314225]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mse_avg    0.113142
Name: sequence_1d_128_9, dtype: object)
('the mean', array([ 2098407.42980906]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mape_avg    2.09841e+06
Name: sequence_1d_128_9, dtype: object)
('the mean', array([ nan]))
('the type of mean', <class 'pandas.core.series.Series'

('the shape of mean', (1,))
("the master df's entry:", mae_avg    0.325776
Name: sequence_2c_289_10_fv1b, dtype: object)
('the mean', array([ 0.32009324]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mse_avg    0.320093
Name: sequence_2c_289_10_fv1b, dtype: object)
('the mean', array([ 1408934.11288103]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mape_avg    1.40893e+06
Name: sequence_2c_289_10_fv1b, dtype: object)
('the mean', array([ nan]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", msle_avg    NaN
Name: sequence_2c_289_10_fv1b, dtype: object)
('the mean', array([ 0.1624856]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mae_avg    0.162486
Name: sequence_1b_68_9_fv1b, dtype: object)
('the mean', array([ 0.0676

("the master df's entry:", mae_avg    0.154784
Name: sequence_2c_249_10_fv1b, dtype: object)
('the mean', array([ 0.06714312]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mse_avg    0.0671431
Name: sequence_2c_249_10_fv1b, dtype: object)
('the mean', array([ 400752.44246777]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mape_avg    400752
Name: sequence_2c_249_10_fv1b, dtype: object)
('the mean', array([ nan]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", msle_avg    NaN
Name: sequence_2c_249_10_fv1b, dtype: object)
('the mean', array([ 0.1608932]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mae_avg    0.160893
Name: sequence_1l_49_10, dtype: object)
('the mean', array([ 0.08530344]))
('the type of mean', <class '

In [6]:
#create a new df: loadseq vs. score
print(master_per_loadseq_df.index)
print(master_per_loadseq_df.columns)
print(master_per_loadseq_df.head(3))

Index([u'sequence_1d_99_10', u'sequence_1d_98_9', u'sequence_1a_18_9',
       u'sequence_1k_179_10', u'sequence_1a_29_10', u'sequence_1d_109_10',
       u'sequence_1a_28_9', u'sequence_1a_19_10', u'sequence_1d_108_9',
       u'sequence_1k_178_9',
       ...
       u'sequence_1b_109_10_fv1b', u'sequence_1b_108_9_fv1b',
       u'sequence_1c_48_9_fv1b', u'sequence_1c_49_10_fv1b',
       u'sequence_2c_288_9_fv1b', u'sequence_2c_289_10_fv1b',
       u'sequence_2c_259_10_fv1b', u'sequence_2c_258_9_fv1b',
       u'sequence_1l_8_9', u'sequence_1l_9_10'],
      dtype='object', name=u'seq_name', length=110)
Index([u'mse_avg', u'mse_med', u'mse_stdev', u'mae_avg', u'mae_med',
       u'mae_stdev', u'mae_normd_dev', u'mape_avg', u'mape_med', u'mape_stdev',
       u'msle_avg', u'msle_med', u'msle_stdev'],
      dtype='object')
                     mse_avg    mse_med mse_stdev   mae_avg    mae_med  \
seq_name                                                                 
sequence_1d_99_10   0.06651

In [7]:
#load the seqname_loadseqlist csv
#print(master)
complete_sn_lsl_df = pd.read_csv(analysis_path + 'complete_seqname_loadseqlist.csv',index_col=0)
complete_sn_lsl_df.index.rename(name='seq_name',inplace=True)
print(complete_sn_lsl_df.index)
print(complete_sn_lsl_df.columns)

Index([u'sequence_2c_239_10_fv1b', u'sequence_1a_78_9',
       u'sequence_1c_9_10_fv1b', u'sequence_1c_38_9_fv1b',
       u'sequence_2c_289_10_fv1b', u'sequence_2b_189_10_fv1b',
       u'sequence_2b_199_10_fv1b', u'sequence_1k_179_10',
       u'sequence_2b_198_9_fv1b', u'sequence_1b_69_10_fv1b',
       ...
       u'sequence_1k_218_9', u'sequence_1l_69_10', u'sequence_1c_29_10_fv1b',
       u'sequence_1k_169_10', u'sequence_2b_218_9_fv1b',
       u'sequence_1b_109_10_fv1b', u'sequence_1b_89_10_fv1b',
       u'sequence_1c_48_9_fv1b', u'sequence_1d_138_9',
       u'sequence_2c_269_10_fv1b'],
      dtype='object', name=u'seq_name', length=110)
Index([u'loadseq_id', u'loadseq'], dtype='object')


In [8]:
#right df is the one described in the cell above. 
seqname_results_loadseq_df = pd.merge(master_per_loadseq_df,complete_sn_lsl_df,how='inner',left_index=True,right_index=True)
print(seqname_results_loadseq_df.head(2))
print(seqname_results_loadseq_df.columns)
print(seqname_results_loadseq_df.index)

                     mse_avg    mse_med mse_stdev   mae_avg    mae_med  \
seq_name                                                                 
sequence_1d_99_10   0.066512  0.0152844   0.10885  0.147706  0.0695016   
sequence_1d_98_9   0.0667542  0.0152675  0.108441  0.147785  0.0698921   

                  mae_stdev mae_normd_dev mape_avg mape_med   mape_stdev  \
seq_name                                                                   
sequence_1d_99_10  0.160956           NaN   578012  11774.2  1.14016e+06   
sequence_1d_98_9   0.160656           NaN   578054  11778.4  1.14026e+06   

                  msle_avg msle_med msle_stdev  loadseq_id  \
seq_name                                                     
sequence_1d_99_10      NaN      NaN        NaN           3   
sequence_1d_98_9       NaN      NaN        NaN           3   

                                                             loadseq  
seq_name                                                              
sequenc

In [9]:
seqname_results_loadseq_df.to_csv(analysis_path+"seqname_score_loadseq_TEST.csv")

In [10]:
columns_to_cast = list(seqname_results_loadseq_df.columns)
columns_to_cast.remove('loadseq')
print(columns_to_cast)
#seqname_results_loadseq_df['mae_avg'].astype(dtype='float64',copy=False)
#pd.to_numeric([columns_to_cast])
for colname in columns_to_cast: #for some reason it's all objects...gotta cast to numeric. 
    seqname_results_loadseq_df[str(colname)] = pd.to_numeric(seqname_results_loadseq_df[str(colname)])
for colname in columns_to_cast:
    print("new", colname, seqname_results_loadseq_df[colname].dtype)
grouped_srls_df = seqname_results_loadseq_df.groupby(by=['loadseq']).mean()
#print(grouped_srls_df['mae_avg'].mean())
#print(grouped_srls_df.median())
#grouped.srls.df.reset_index()
print(grouped_srls_df.head(29))
print(type(grouped_srls_df))
#print(type(grouped_srls_df.groups))
#print(grouped_srls_df.head(1))#print(grouped_srls_df.mae_avg)
#print(grouped_srls_df.agg(np.mean))
#for key,value in grouped_srls_df.groups.iteritems():
    #print(value)
#print(grouped_srls_df.head(2))
pd.DataFrame(grouped_srls_df).to_csv(analysis_path+'srls_grouped_by_ls.csv')

['mse_avg', 'mse_med', 'mse_stdev', 'mae_avg', 'mae_med', 'mae_stdev', 'mae_normd_dev', 'mape_avg', 'mape_med', 'mape_stdev', 'msle_avg', 'msle_med', 'msle_stdev', 'loadseq_id']
('new', 'mse_avg', dtype('float64'))
('new', 'mse_med', dtype('float64'))
('new', 'mse_stdev', dtype('float64'))
('new', 'mae_avg', dtype('float64'))
('new', 'mae_med', dtype('float64'))
('new', 'mae_stdev', dtype('float64'))
('new', 'mae_normd_dev', dtype('float64'))
('new', 'mape_avg', dtype('float64'))
('new', 'mape_med', dtype('float64'))
('new', 'mape_stdev', dtype('float64'))
('new', 'msle_avg', dtype('float64'))
('new', 'msle_med', dtype('float64'))
('new', 'msle_stdev', dtype('float64'))
('new', 'loadseq_id', dtype('int64'))
                                                     mse_avg   mse_med  \
loadseq                                                                  
[  1.028  10.28   10.28    0.75    0.75   10.28...  0.070108  0.018783   
[  1.028  11.28   11.28    1.      1.     10.28...  0.142666 

In [11]:
grouped_srls_count_df = seqname_results_loadseq_df.groupby(by=['loadseq']).count()
pd.DataFrame(grouped_srls_count_df).to_csv(analysis_path+'srls_grouped_by_ls_count.csv')

In [12]:
#group by initial conditions. 
